# install and import packages

In [4]:
!pip install pyDOE2 scikit-learn lime anchor-exp

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 KB 2.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 KB 5.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 7.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 8.0 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 KB 5.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 7.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.9/221.9 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.1 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 KB 5.7 MB/s eta 0:00:00
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283855 sha256=51da38d946feb40cb5ad9a53e1b5cbb561926649cb883602e0c75c24ed708b97
  Stored in directory: /home/genji/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
  Created wheel for anchor-exp: filename=anchor_exp-0.0.2.0-py3-none-any.whl size=433523 sha256=f55b4d0d8472e1219dcc2129c140d16a70541314d5980386c12d6bf40915aa77
  Stored in directory: /home/genji/.cache/pip/wheels/e9/53/45/7e4602020c5e5069ccef79f1389adb8efc4ca3c4d9891388bb
Successfully built lime anchor-exp
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
 

In [5]:
# from google.colab import drive
# drive.mount('/content/drive/')

# import sys
# sys.path.append('/content/drive/MyDrive/Programs/Lab')

In [6]:
import sklearn
import sklearn.datasets
import sklearn.ensemble
import numpy as np

# LIME
import explanation
import lime_tabular

# Anchor
import utils
import anchor_tabular

import importlib
importlib.reload(explanation)
importlib.reload(lime_tabular)

<module 'lime_tabular' from '/home/genji/GoogleDrive/Programs/Lab/lime_tabular.py'>

# Iris

### Preprocess dataset

In [ ]:
iris = sklearn.datasets.load_iris()
train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(
    iris.data, iris.target, train_size=0.80)

### Learn blackbox model and generate explanations

In [ ]:
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
rf.fit(train, labels_train)
sklearn.metrics.accuracy_score(labels_test, rf.predict(test))

In [ ]:
explainer = lime_tabular.LimeTabularExplainer(
    train,
    feature_names=iris.feature_names,
    class_names=iris.target_names,
    discretize_continuous=True)

i = np.random.randint(0, test.shape[0])

exp = explainer.explain_instance(
    test[i], rf.predict_proba, num_features=2, top_labels=1)

exp.show_in_notebook(show_table=True, show_all=True)

# Adult

## Preprocess the dataset

In [ ]:
# make sure you have adult/adult.data inside dataset_folder
dataset_folder = '/content/drive/MyDrive/Programs/Lab/datasets/'
dataset = utils.load_dataset(
    'adult', balance=True, dataset_folder=dataset_folder, discretize=True)

## Learn blackbox model and generate explanations

In [ ]:
# make sure you have adult/adult.data inside dataset_folder
dataset_folder = '/content/drive/MyDrive/Programs/Lab/datasets/'
dataset = utils.load_dataset(
    'adult', balance=True, dataset_folder=dataset_folder, discretize=False)

In [ ]:
rf.fit(dataset.train, dataset.labels_train)
print('Train', sklearn.metrics.accuracy_score(
    dataset.labels_train, rf.predict(dataset.train)))
print('Test', sklearn.metrics.accuracy_score(
    dataset.labels_test, rf.predict(dataset.test)))

In [ ]:
lime_explainer = lime_tabular.LimeTabularExplainer(
    dataset.train,
    feature_names=dataset.feature_names,
    class_names=dataset.class_names,
    discretize_continuous=False)

anchor_explainer = anchor_tabular.AnchorTabularExplainer(
    dataset.class_names,
    dataset.feature_names,
    dataset.train,
    dataset.categorical_names)

In [ ]:
i = np.random.randint(0, dataset.test.shape[0])

trg = dataset.test[i]

lime_exp = lime_explainer.explain_instance(
    trg, rf.predict_proba, num_features=5, top_labels=1)

lime_exp.show_in_notebook(show_table=True, show_all=True)

In [ ]:
anchor_exp = anchor_explainer.explain_instance(trg, rf.predict, threshold=0.95)

print('Prediction:', anchor_explainer.class_names[rf.predict(trg.reshape(1, -1))[0]])
print('Anchor:     %s' % (' AND '.join(anchor_exp.names())))
print('Precision:  %.2f' % anchor_exp.precision())
print('Coverage:   %.2f' % anchor_exp.coverage())

In [ ]:
# Get test examples where the anchor applies
fit_anchor = np.where(np.all(
    dataset.test[:, anchor_exp.features()] == trg[anchor_exp.features()],axis=1))[0]

same_label = lambda x : rf.predict(dataset.test[x]) == rf.predict(trg.reshape(1, -1))

print('Anchor test precision: %.2f' % (np.mean(same_label(fit_anchor))))
print('Anchor test coverage: %.2f' % (
    fit_anchor.shape[0] / float(dataset.test.shape[0])))